In [21]:
import os
import pandas as pd
import sqlite3

In [22]:
# Define the SQLite database path
db_path = 'C:/Learning/Taads 1/courses01.sqlite'

# Define the folder containing the dataset files
data_folder = 'C:/Learning/tads/Courses/other_courses'  # Update to your folder containing all datasets

In [23]:
# Function to clean a dataset
def clean_dataset(df):
    # Fill missing values
    df.fillna("Not specified", inplace=True)

    # Standardize column names
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]

    # Handle invalid prices or negative values if relevant columns exist
    if 'price' in df.columns and 'num_subscribers' in df.columns:
        df = df[(df['price'] >= 0) & (df['num_subscribers'] >= 0)]
    return df

In [24]:
# Function to add a dataset to the SQLite database
def add_to_sqlite(df, table_name, db_path):
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

In [25]:
# Loop through all files in the data folder
for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)

    # Read the dataset based on file extension
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        print(f"Unsupported file format: {file_name}")
        continue

    # Clean the dataset
    df_cleaned = clean_dataset(df)

    # Generate a table name based on the file name
    table_name = os.path.splitext(file_name)[0].lower().replace(" ", "_")

    # Add the cleaned dataset to the SQLite database
    add_to_sqlite(df_cleaned, table_name, db_path)

    print(f"Added {file_name} as table {table_name} in SQLite database.")

Added course-catalog.csv as table course-catalog in SQLite database.
Added coursera.csv as table coursera in SQLite database.


C:\Users\gaura\AppData\Local\Temp\ipykernel_8208\846106797.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not specified' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Not specified", inplace=True)


Added Coursera_2.csv as table coursera_2 in SQLite database.
Added CoursesData.csv as table coursesdata in SQLite database.
Added EdX.csv as table edx in SQLite database.
Added udemy_courses.csv as table udemy_courses in SQLite database.


In [26]:
# Verify the tables in the SQLite database
conn = sqlite3.connect(db_path)
created_tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
conn.close()

print("Tables in the database:", created_tables)

Tables in the database: [('cleaned_courses',), ('course-catalog',), ('coursera',), ('coursera_2',), ('coursesdata',), ('edx',), ('udemy_courses',)]


In [27]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [28]:
import sqlite3
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import os
import numpy as np

In [29]:
# Function to load data from all tables in SQLite database
def load_data_from_all_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get all table names from the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    documents = []
    for table_name in tables:
        table_name = table_name[0]  
        print(f"Loading data from table: {table_name}")

        # Fetch all rows from the table
        cursor.execute(f'SELECT * FROM "{table_name}"')
        rows = cursor.fetchall()

        # Combine rows into Document objects
        for row in rows:
            page_content = " ".join(map(str, row[1:]))  
            documents.append(Document(page_content=page_content))

    conn.close()
    return documents

In [30]:
# Define the database path
db_path = 'C:/Learning/Taads 1/courses01.sqlite'



In [31]:
# Load data from all tables
documents = load_data_from_all_tables(db_path)

Loading data from table: cleaned_courses
Loading data from table: course-catalog
Loading data from table: coursera
Loading data from table: coursera_2
Loading data from table: coursesdata
Loading data from table: edx
Loading data from table: udemy_courses


In [32]:
# Split documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
final_documents = text_splitter.split_documents(documents)

# Verify loading and splitting
print(final_documents[0])
print("Total number of document chunks:", len(final_documents))

▪ Self-study There are no formal requirements. Nieuwenhuis, P. / Wells, P.: Global Automotive Industry, John Wiley & Sons, 2015\nMunson, C.: The Supply Chain Management Casebook, FT Press, 2013Diehlmann, J. / Häcker, J.: Automotive Management, 2nd ed., Oldenbourg Verlag, 2013 Myerson, P. A.: Lean and Technology: Working Hand in Hand to Enable and Energize Your Global Supply Chain, Pearson Education, 2017F urther references will be given during the classes. This course is in particular applicable to the following Master programmes: International Business and Economics (M.A.; “IBE”).
▪ Readings and exam preparation: 54 6  Comprehensive written examination, 90 minutes (100%) First academic year Summer 1 Semester Elective English b'`\xe4\\<\xf6\xb4\x00=\xad▪ Self-study There are no formal requirements. Nieuwenhuis, P. / Wells, P.: Global Automotive Industry, John Wiley & Sons, 2015\nMunson, C.: The Supply Chain Management Casebook, FT Press, 2013Diehlmann, J. / Häcker, J.: Automotive Man

In [33]:
# Initialize embeddings using HuggingFace
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Example embedding for verification
embedding_example = np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))
print("Embedding vector for the first document chunk:", embedding_example)
print("Shape of the embedding vector:", embedding_example.shape)

Embedding vector for the first document chunk: [-1.20745664e-02  7.87300915e-02  3.50556485e-02 -3.51754809e-03
  4.15791385e-02  1.76234320e-02 -2.72780168e-03  7.31812119e-02
 -7.47286305e-02 -6.48280084e-02  3.67245264e-02 -3.15500125e-02
  1.89453989e-04 -8.75325967e-03  1.52374785e-02  1.68769285e-02
  3.09608579e-02  6.90319994e-03  5.20508550e-02 -7.78148025e-02
 -1.28828995e-02 -4.48709391e-02 -5.97063685e-03 -1.52580449e-02
  5.33556491e-02  1.23097850e-02  2.62966733e-02  2.56084464e-02
 -2.69088447e-02 -1.89657003e-01 -5.13429157e-02  2.75295377e-02
  1.83006544e-02  6.51086718e-02  9.47159156e-03  3.43765505e-02
 -7.25370124e-02 -9.51696653e-03  9.83149931e-03 -5.51961772e-02
 -4.87998361e-03  4.15198691e-03  1.42298974e-02 -7.33447596e-02
  1.63218360e-02 -3.56145911e-02  3.48472525e-03 -2.20933743e-02
 -1.34164281e-02 -6.70476109e-02 -1.20900460e-02 -5.57899326e-02
  1.56254359e-02 -4.86149751e-02 -1.90979429e-02  4.46648374e-02
  2.90481318e-02  1.15750711e-02 -1.9045185

In [34]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [35]:
# Create FAISS VectorStore for similarity search
vectorstore = FAISS.from_documents(final_documents, huggingface_embeddings)

# Example Query Using Similarity Search
query = "I want to study mobile systems"
relevant_documents = vectorstore.similarity_search(query)

# Display content of the most relevant document found
print("Most relevant document content:\n", relevant_documents[0].page_content)

# Set up the retriever with similarity search configuration
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
print("Retriever set up successfully:", retriever)

Most relevant document content:
Greg Milette, Adam Stroud, Professional Android Sensor Programming, John Wiley, Indianapolis 2012 Master of Applied Computer Science 150 hours: 60 hours presence, 45 hours self-study, 45 hours preparation of exam 5  Written exam 2nd semester Winter 1 Semester Obligatory English b'\x05\xfd\x1e\xbd\x04~4=\xed\xc1!\xbcw?\xff\xbdh\xdb\x84;\xa2=\xd2\xbcO\xb0\x08=2\xdb!>H\x8d\x0f=\x85\xb0\x8c=\x18\x96Y=\xac*\xe6<K\x0c\n>g4\x1a\xbd\x93\x06\x18=\xcd\xeb\x8b\xbd\xe0\xf5\x05=\x99\xa8*\xbdP\x00\xf0\xbc\xdb\xa9\x84;\xcf~\x13=*a\x08=\xc8\xff\x1d\xbef\x88\x82:M3\xb3;~\xae\xc4\xbc\x84\xb4:\xbd\r\xa2\x12\xbd\x18\x05\x07<\xffd\x98\xbd\xacl#\xbd\xf7\xd7\x8b=<*\xbf=8.\x18=\xb9\xedv\xbd\xbbz\xbf\xbdy\xc6\xc2=\x84\x94\x80\xbd\x85\xe08\xbd\xafU\xad\xbc\xf7[\xe9\xbdd\xc6-:\x84\xe3\xe5<\xe5\x19\x9a={\x7f\xa0\xbc\xb0\x12+<\x7f4\x8a;\x85\x81\x93:\x06\x82\x9a\xbb\xf8\xba\x81\xbd\x12\x90\xb2<\x13TL\xbd\xc1@\x13\xbcz\xdb#=\xdd1\xa5\xbdQ{\x9b;d\xfa\xb1\xbbCw\xd5=_\xd6\xe9<<\xa4?\xbcl

In [36]:
# Hugging Face API setup 
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "YOUR_TOKEN_HERE"

# Initialize Hugging Face model 
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={"temperature": 0.8, "max_length": 300}
)

# Execute a query to get a response
response = hf.invoke(query)
print("Response from Hugging Face model:\n", response)

Response from Hugging Face model:
 I want to study mobile systems and realtime systems. Where do I start?

There are many resources available for studying mobile systems and realtime systems. Here are some suggestions:

1. Textbooks:
* Operating Systems: Three Easy Pieces by remi forax and Andrew S. Tanenbaum
* Real-Time Systems: Principles and Techniques by John C. Tsichydis and George D. Tsichydis
* Mobile Computing: Principles and Practice by Olukotun and Shroff
* Computer Networks: A Systems Approach by Kurose and Ross
2. Online Courses:
* Coursera: "Mobile Cloud Computing" by University of California, Irvine
* edX: "Real-Time and Embedded Systems" by University of Illinois at Urbana-Champaign
* MIT OpenCourseWare: "Operating System Engineering" and "6.824: Mobile Systems Engineering"
3. Research papers:
* ACM Digital Library: Search for papers on mobile systems and real-time systems
* IEEE Xplore Digital Library: Search for papers on mobile systems and real-time systems
4. Open so

In [37]:
prompt_template = """
Your answer must be based solely on the context provided below. Don't give irrelevant information.

Context:
{context}

Question:
{question}

Answer:
"""

In [38]:
from langchain import PromptTemplate
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

from langchain.chains import RetrievalQA
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [39]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [40]:
from flask import Flask, request, jsonify, Response
from werkzeug.serving import run_simple
import traceback

app = Flask(__name__)

# In-memory storage for chat history
chat_history = []

# Define the search_query function
def search_query(query):
    try:
        # Call the QA chain with the query
        result = retrievalQA.invoke({"query": query})

        # Directly extract the response content
        full_response = result['result']  # Retrieve the generated result from the chain

        # Find the "Answer:" portion and isolate the actual answer
        if "Answer:" in full_response:
            answer_start = full_response.find("Answer:") + len("Answer:")
            answer = full_response[answer_start:].strip()
        else:
            # Fallback if "Answer:" label is missing
            answer = full_response.strip()

        # Store the query and answer in chat history
        chat_history.append({"query": query, "answer": answer})

        # Format the output with Question and Answer only
        return f"Query:\n{query}\n\nAnswer:\n{answer}"
    except Exception as e:
        # Handle errors gracefully
        return f"An error occurred: {e}"

@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Course Campus</title>
        <style>
            /* Global Styles */
            body {
                background: #f8f4ec;
                color: #2d2d2d;
                font-family: 'Roboto', sans-serif;
                margin: 0;
                padding: 0;
            }

            h1, p {
                margin: 0;
            }

            #title-section {
                background: #d6eae5;
                padding: 20px;
                border-radius: 12px;
                text-align: center;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                margin: 20px auto;
                width: 90%;
                max-width: 800px;
            }

            #title-section h1 {
                font-size: 2.4em;
                color: #2a3d34;
                text-transform: uppercase;
            }

            #title-section p {
                font-size: 1.1em;
                font-style: italic;
                color: #495d54;
            }

            .container {
                width: 90%;
                max-width: 800px;
                margin: auto;
                padding: 20px;
                background: #ffffff;
                border-radius: 12px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }

            label {
                font-size: 1em;
                margin-bottom: 8px;
                display: block;
                color: #2d2d2d;
            }

            input[type="text"], textarea {
                width: 100%;
                padding: 12px;
                border: 1px solid #ccc;
                border-radius: 8px;
                margin-bottom: 20px;
                font-size: 1em;
                background: #f8f8f8;
                color: #333;
                box-shadow: inset 0 2px 4px rgba(0, 0, 0, 0.05);
            }

            input:focus, textarea:focus {
                border-color: #469d89;
                outline: none;
                background: #ffffff;
            }

            button {
                width: 100%;
                padding: 12px;
                font-size: 1.1em;
                color: #ffffff;
                background: #469d89;
                border: none;
                border-radius: 8px;
                cursor: pointer;
                transition: all 0.3s ease-in-out;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }

            button:hover {
                background: #3a8575;
                box-shadow: 0 6px 12px rgba(0, 0, 0, 0.15);
            }

            textarea {
                resize: none;
                height: 150px;
                font-size: 1.1em; 
            }

            .output-box {
                background: #f8f8f8;
                color: #333;
            }

            .history-section {
                margin-top: 20px;
                background: #e8f1f7;
                padding: 15px;
                border-radius: 8px;
            }

            .history-section ul {
                list-style: none;
                padding: 0;
            }

            .history-section li {
                margin-bottom: 10px;
                padding: 10px;
                background: #ffffff;
                border-radius: 6px;
                box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            }

            /* Footer */
            footer {
                text-align: center;
                padding: 10px 0;
                margin-top: 20px;
                color: #777;
                font-size: 0.9em;
            }

            /* Media Queries */
            @media (max-width: 768px) {
                #title-section {
                    width: 95%;
                }
                .container {
                    width: 95%;
                }
                button {
                    font-size: 1em;
                }
            }
        </style>
    </head>
    <body>
        <div id="title-section">
            <h1>Course Campus</h1>
            <p>Your guide to learning opportunities</p>
        </div>
        <div class="container">
            <div>
                <label for="query">Query:</label>
                <input type="text" id="query" class="input-box" placeholder="Enter your query">
            </div>
            <button onclick="search()">Search</button>
            <div>
                <label for="results">Output:</label>
                <textarea id="results" class="output-box" readonly></textarea>
            </div>
            <div class="history-section">
                <h2>Chat History</h2>
                <ul id="history"></ul>
                <button onclick="downloadHistory()">Download Chat History</button>
            </div>
        </div>
        <footer>
            &copy; 2025 Course Campus. All Rights Reserved.
        </footer>
        <script>
            async function search() {
                const query = document.getElementById('query').value;
                const resultsBox = document.getElementById('results');

                if (!query) {
                    resultsBox.value = 'Please enter your query.';
                    return;
                }

                resultsBox.value = 'Searching...';

                try {
                    const response = await fetch('/search', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({ query })
                    });

                    const data = await response.json();

                    if (data.error) {
                        resultsBox.value = `Error: ${data.error}`;
                    } else {
                        resultsBox.value = data.result;
                        addToHistory(query, data.result);
                    }
                } catch (error) {
                    resultsBox.value = `An error occurred: ${error.message}`;
                }
            }

            function addToHistory(query, answer) {
                const historyList = document.getElementById('history');
                const listItem = document.createElement('li');
                listItem.textContent = `Query: ${query} | Answer: ${answer}`;
                historyList.appendChild(listItem);
            }

            function downloadHistory() {
                fetch('/download-history')
                    .then(response => response.blob())
                    .then(blob => {
                        const url = window.URL.createObjectURL(blob);
                        const a = document.createElement('a');
                        a.style.display = 'none';
                        a.href = url;
                        a.download = 'chat_history.txt';
                        document.body.appendChild(a);
                        a.click();
                        window.URL.revokeObjectURL(url);
                    })
                    .catch(error => alert('Failed to download history: ' + error));
            }
        </script>
    </body>
    </html>
    '''

@app.route('/search', methods=['POST'])
def search():
    try:
        data = request.json
        query = data.get("query", "")
        if not query:
            return jsonify({"error": "Query cannot be empty"}), 400

        result = search_query(query)
        return jsonify({"result": result})
    except Exception as e:
        return jsonify({"error": f"An error occurred: {e}", "trace": traceback.format_exc()}), 500

@app.route('/download-history', methods=['GET'])
def download_history():
    try:
        history_content = "\n".join([f"Query: {item['query']}\nAnswer: {item['answer']}\n" for item in chat_history])
        return Response(history_content, mimetype='text/plain', headers={"Content-Disposition": "attachment;filename=chat_history.txt"})
    except Exception as e:
        return jsonify({"error": f"An error occurred: {e}", "trace": traceback.format_exc()}), 500

if __name__ == '__main__':
    run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jan/2025 14:12:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:16:29] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:16:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:20:09] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:20:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:27:46] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:27:55] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:05] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:10] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:29] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:28:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 14:29:06] "POST /search HTT